# Aprendizaje Multietiqueta de Patrones Geométricos en Objetos de Herencia Cultural
# Kunisch Features from ResNet architectures
## Seminario de Tesis II, Primavera 2022
### Master of Data Science. Universidad de Chile.
#### Prof. guía: Benjamín Bustos - Prof. coguía: Iván Sipirán
#### Autor: Matías Vergara


## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
import time
import os
import copy
import pandas as pd
import math
import random
import shutil

from torch.utils.data import Dataset
from PIL import Image

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np, scipy.io
import argparse
import json

## Mounting Google Drive

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    folder_path = 'drive/MyDrive/TesisMV/'
except:
    folder_path = '../'

In [3]:
os.listdir(folder_path)

['.git',
 '.gitignore',
 '.idea',
 'actas',
 'features',
 'graficos',
 'labels',
 'models',
 'notebooks',
 'patterns',
 'presentaciones',
 'README.md',
 'requirements.txt',
 'tablas']

## Dataset and model selection

In [4]:
#modify only this cell
USE_RN50 = True
SUBCHAPTERS = False
DS_FLAGS = ['blur']
              # 'ref': [invertX, invertY],
              # 'rot': [rotate90, rotate180, rotate270],
              # 'crop': [crop] * CROP_TIMES,
              # 'blur': [blur],
              # 'emboss': [emboss],
              # 'randaug': [randaug],
              # 'rain': [rain],
              # 'elastic': [elastic]
CROP_TIMES = 1
RANDOM_TIMES = 2
ELASTIC_TIMES = 1

In [5]:
# This cells builds the data_flags variable, that will be used
# to map the requested data treatment to folders
MAP_TIMES = {'crop': CROP_TIMES,
         'randaug': RANDOM_TIMES,
         'elastic': ELASTIC_TIMES,
}

DS_FLAGS = sorted(DS_FLAGS)
data_flags = '_'.join(DS_FLAGS) if len(DS_FLAGS) > 0 else 'base'
MULTIPLE_TRANSF = ['crop', 'randaug', 'elastic']
COPY_FLAGS = DS_FLAGS.copy()

for t in MULTIPLE_TRANSF:
    if t in DS_FLAGS:
        COPY_FLAGS.remove(t)
        COPY_FLAGS.append(t + str(MAP_TIMES[t]))
        data_flags = '_'.join(COPY_FLAGS)

subchapter_str = 'subchapters/' if SUBCHAPTERS else ''
patterns_path = folder_path + 'patterns/' + subchapter_str + data_flags + "/"
labels_path = folder_path + 'labels/' +  subchapter_str + data_flags + "/"
# models_path = folder_path + 'models/' + subchapter_str + (f'resnet50_{data_flags}.pth' if USE_RN50 else f'resnet18_{data_flags}.pth')
# features_path = folder_path + 'features/' + subchapter_str + (f'resnet50_{data_flags}/' if USE_RN50 else f'resnet18_{data_flags}/')

rn = 18
ep = 65
models_path = folder_path + f"models/resnet{rn}_blur_each5/resnet{rn}_blur_e{ep}.pth"
features_path = folder_path + f"features/resnet{rn}_blur_each5/resnet{rn}_blur_e{ep}/"

if not (os.path.isdir(patterns_path) and os.path.isdir(labels_path)):
    raise FileNotFoundError("No existen directorios de datos para el conjunto de flags seleccionado. Verifique que el dataset exista y, de lo contrario, llame a Split and Augmentation")

if not (os.path.isfile(models_path)):
    raise FileNotFoundError(f"No se encontró modelo para el conjunto de flags seleccionado. Verifique que el modelo exista y, de lo contrario, llame a ResNet Retraining\n{models_path}")


print("Pattern set encontrado en {}".format(patterns_path))
print("Labels set encontrado en {}".format(labels_path))
print("Modelo encontrado en {}".format(models_path))

Pattern set encontrado en ../patterns/blur/
Labels set encontrado en ../labels/blur/
Modelo encontrado en ../models/resnet18_blur_each5/resnet18_blur_e65.pth


## Dataset loader

In [6]:
class PatternDataset(Dataset):
    def __init__(self, root_dir, transform=None, build_classification=False, name_cla='output.cla'):
        self.root_dir = root_dir
        self.transform = transform
        self.namefiles = []


        self.classes = sorted(os.listdir(self.root_dir))

        for cl in self.classes:
            for pat in os.listdir(os.path.join(self.root_dir, cl)):
                self.namefiles.append((pat, cl))

        print(f'Files:{len(self.namefiles)}')
        self.namefiles = sorted(self.namefiles, key = lambda x: x[0])

        if build_classification:
            dictClasses = dict()

            for cl in self.classes:
                dictClasses[cl] = []

            for index, (name, cl) in enumerate(self.namefiles):
                dictClasses[cl].append((name, index))

            with open(name_cla, 'w') as f:
                f.write('PSB 1\n')
                f.write(f'{len(self.classes)} {len(self.namefiles)}\n')
                f.write('\n')
                for cl in self.classes:
                    f.write(f'{cl} 0 {len(dictClasses[cl])}\n')
                    for item in dictClasses[cl]:
                        f.write(f'{item[1]}\n')
                    f.write('\n')

    def __len__(self):
        return len(self.namefiles)

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()

        img_name = os.path.join(self.root_dir, self.namefiles[index][1], self.namefiles[index][0])
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return self.namefiles[index], image


## Preparation

In [7]:
random.seed(30)

output_train = features_path + "augmented_train_df.json"
output_val = features_path + "val_df.json"
output_test = features_path + "test_df.json"

train_df = pd.read_json(labels_path + "augmented_train_df.json", orient='index')
val_df = pd.read_json(labels_path + "val_df.json", orient='index')
test_df = pd.read_json(labels_path + "test_df.json", orient='index')

train_pts = train_df.index.values
val_pts = val_df.index.values
test_pts = test_df.index.values

## Extraction

In [8]:
def imshow(inp, title = None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)

    plt.imshow(inp)
    plt.show()

def get_vector(model,layer, dim_embedding, x):

  my_embedding = torch.zeros(dim_embedding)

  def copy_data(m,i,o):
    my_embedding.copy_(o.data.squeeze())

  h = layer.register_forward_hook(copy_data)
  model(x)
  h.remove()

  return my_embedding

device = ('cuda' if torch.cuda.is_available() else 'cpu')

my_transform = transforms.Compose([ transforms.Resize(224),
                                    #transforms.CenterCrop(224),
                     transforms.ToTensor(),
                     transforms.Normalize(mean=[0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])
                    ])

dataTrain = PatternDataset(root_dir=os.path.join(patterns_path, 'train'), transform=my_transform)
dataVal = PatternDataset(root_dir=os.path.join(patterns_path, 'val'), transform=my_transform)
dataTest = PatternDataset(root_dir=os.path.join(patterns_path, 'test'), transform=my_transform)

loaderTrain = DataLoader(dataTrain)
loaderVal = DataLoader(dataVal)
loaderTest = DataLoader(dataTest)

if USE_RN50:
    model = models.resnet50(pretrained = True)
else:
    model = models.resnet18(pretrained = True)
dim = model.fc.in_features

output_dim = 20 if SUBCHAPTERS else 6
model.fc = nn.Linear(dim, output_dim)

model = model.to(device)

try:
    model.load_state_dict(torch.load(models_path))
except RuntimeError as e:
    print('Ignoring "' + str(e) + '"')

layer = model._modules.get('avgpool')

model.eval()

#features = []
features_train = {}
features_val = {}
features_test = {}


for name, img in loaderTrain:
  feat = get_vector(model, layer, dim, img.to(device))
  namefile = name[0][0]
  code, rest = namefile.split('.')
  features_train[code] = feat.numpy().tolist()
  #features.append(feat.numpy())

for name, img in loaderVal:
  feat = get_vector(model, layer, dim, img.to(device))
  namefile = name[0][0]
  code, rest = namefile.split('.')
  features_val[code] = feat.numpy().tolist()
  #features.append(feat.numpy())

for name, img in loaderTest:
  feat = get_vector(model, layer, dim, img.to(device))
  namefile = name[0][0]
  code, rest = namefile.split('.')
  features_test[code] = feat.numpy().tolist()
#features = np.vstack(features)
#print(features.shape)

Files:1084
Files:78
Files:156
Ignoring "Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "layer1.0.conv3.weight", "layer1.0.bn3.weight", "layer1.0.bn3.bias", "layer1.0.bn3.running_mean", "layer1.0.bn3.running_var", "layer1.0.downsample.0.weight", "layer1.0.downsample.1.weight", "layer1.0.downsample.1.bias", "layer1.0.downsample.1.running_mean", "layer1.0.downsample.1.running_var", "layer1.1.conv3.weight", "layer1.1.bn3.weight", "layer1.1.bn3.bias", "layer1.1.bn3.running_mean", "layer1.1.bn3.running_var", "layer1.2.conv1.weight", "layer1.2.bn1.weight", "layer1.2.bn1.bias", "layer1.2.bn1.running_mean", "layer1.2.bn1.running_var", "layer1.2.conv2.weight", "layer1.2.bn2.weight", "layer1.2.bn2.bias", "layer1.2.bn2.running_mean", "layer1.2.bn2.running_var", "layer1.2.conv3.weight", "layer1.2.bn3.weight", "layer1.2.bn3.bias", "layer1.2.bn3.running_mean", "layer1.2.bn3.running_var", "layer2.0.conv3.weight", "layer2.0.bn3.weight", "layer2.0.bn3.bias", "layer2.0.bn3.runn

In [9]:
os.makedirs(features_path, exist_ok=True)

In [10]:
features_train_df = pd.DataFrame.from_dict(features_train, orient='index')
features_val_df = pd.DataFrame.from_dict(features_val, orient='index')
features_test_df = pd.DataFrame.from_dict(features_test, orient='index')

features_train_df.to_json(output_train, orient='index')
features_val_df.to_json(output_val, orient='index')
features_test_df.to_json(output_test, orient='index')

In [11]:
features_train_df

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
10d,0.118115,0.016901,0.050756,0.018998,0.072170,1.027323,1.086565,1.007553,0.000703,0.013933,...,0.005300,0.002009,0.000954,1.300107,1.786549,2.534481,0.007738,0.214728,0.639668,0.024192
10d_blur,0.127972,0.018648,0.054972,0.016804,0.067858,0.994569,1.073645,1.013490,0.000683,0.014963,...,0.005495,0.002043,0.001085,1.324607,1.807548,2.523198,0.007684,0.228314,0.668426,0.031063
10e,0.108879,0.027604,0.031459,0.017958,0.085115,1.127079,1.093070,0.928611,0.001364,0.015925,...,0.008093,0.002688,0.003541,1.174742,1.662527,2.533029,0.006802,0.174143,0.512420,0.025664
10e_blur,0.126336,0.022580,0.047373,0.016586,0.074275,1.091521,1.049026,0.985725,0.000764,0.015809,...,0.006928,0.002119,0.002332,1.289737,1.753891,2.544923,0.006608,0.247875,0.618404,0.028839
11c,0.157164,0.057202,0.040079,0.028363,0.100762,1.146053,1.053323,0.978663,0.008755,0.092198,...,0.010211,0.011056,0.003314,1.247845,1.764787,2.339533,0.003815,0.299079,0.584380,0.032056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9b_blur,0.136078,0.025349,0.056788,0.020624,0.074448,1.029460,1.048925,1.029510,0.000893,0.027187,...,0.007051,0.003334,0.001598,1.287908,1.835416,2.476223,0.008846,0.236722,0.676854,0.030232
9c,0.130970,0.024469,0.051212,0.018738,0.075215,1.041094,1.059446,1.045548,0.001118,0.021973,...,0.006225,0.003102,0.002008,1.281846,1.791671,2.490650,0.008477,0.234247,0.675624,0.027513
9c_blur,0.137362,0.024895,0.056910,0.020012,0.073393,1.005419,1.056477,1.034944,0.000994,0.027053,...,0.006680,0.003199,0.001427,1.315596,1.833452,2.467321,0.007377,0.238437,0.672983,0.030981
9e,0.112205,0.041844,0.031186,0.020477,0.108583,1.101948,1.083312,0.928882,0.002444,0.029543,...,0.009869,0.005491,0.005958,1.095611,1.619138,2.480907,0.008249,0.173625,0.523920,0.027280
